# 環境設定

## 安裝 GPU 驅動相關套件

In [ ]:
# 更新 Linux 內的套件清單至最新版
!apt-get update

# 安裝 PCI 匯流排工具（PCI Utility）與 lshw (LiSt HardWare)，以便能偵測到 GPU
# -y：遇到詢問是否安裝，一律自動回答 yes
!apt-get install -y pciutils lshw

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,167 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,424 kB in 1s (2,524 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done

In [ ]:
# 用 nVidia 的 System Management Interface (SMI) 確認 GPU 的確抓得到
!nvidia-smi

Tue Jun 11 01:49:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 安裝 Ollama

In [ ]:
# 至 https://ollama.com/download/linux
# 直接將安裝 Ollama 於 Linux 的指令貼上
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# 啟動 Ollama，讓它執行於背景中
# ollama serve: 用 Server 模式、而非互動模式執行 Ollama
# > server.log：將本應顯示於螢幕的訊息，轉向輸出至 server.log 這個檔備查
# 2>&1：2 為 stderr。將所有錯誤訊息，轉向 &1 (stdout，螢幕) 輸出。
# &：將程式啟動之後，馬上返回，不要等該程式執行完成
!ollama serve > server.log 2>&1 &

In [ ]:
# 將 Llama-3 模型下載，並做為此次的大語言模型
# ollama run <模型名稱>：下載並執行特定 LLM
# > model.log：將本應顯示於螢幕的訊息，轉向輸出至 model.log 這個檔備查
!ollama run gemma:7b > model.log 2>&1 &

# 注意：上述兩指令皆以「&」後綴，告知 Colab「不用等 Linux 執行完」。
# 但事實上，不論啟動為 Server，或下載 LLM，皆須 1~5 分鐘不等的時間。
# 可以查看 server.log、model.log 兩檔案內容，得知當前執行狀況。

## 安裝 LangChain + 連上大語言模型

In [ ]:
# 下載 LangChain，一套專門連上各種大語言模型的 Python 套件
!pip install langchain  # LangChain 核心元件
!pip install langchain-community  # 各種開源大語言模型連接函數套件

In [ ]:
# 載入 Ollama 以便連上後端 LLM
from langchain_community.llms import Ollama

# LLM 名稱需與 ollama run 後方名稱相同
llm = Ollama(model="gemma:7b")

In [ ]:
# 提供 LLM 一個「系統角色設定」，並給予一個應答範例
system_prompt = """
### 角色設定

你現在是一個早餐店的櫃臺服務員。你的主要任務是接待客戶，提供菜單，聽取並複述客戶的點單內容，並計算總金額。請根據以下指示操作：

### 任務描述

1. **打招呼**
   - 當客戶輸入「早安」、「你好」或類似的問候語且沒有進一步資訊時，請回答：「早安！請問今天要吃什麼？」

2. **提供菜單**
   - 當客戶提到「菜單」、「menu」或詢問「這邊有什麼好吃的」、「你推薦一些好吃的給我」時，請回答：「這邊有我們的菜單，請你參考看看。」並附上以下菜單：
     - 漢堡蛋：NT$50
     - 總匯三明治：NT$70
     - 奶茶：NT$15

3. **複述客戶點單內容**
   - 當客戶下單時，聽取他們的點單內容並按照以下格式複述：
     ```
     <產品1> x <數量>
     <產品2> x <數量>
     ```
   - 例如，客戶說：「我要兩個漢堡蛋，一杯奶茶，和一個總匯三明治」，你的回應應為：
     ```
     漢堡蛋 x2
     總匯三明治 x1
     奶茶 x1
     ```

4. **計算總金額**
   - 根據菜單價格和客戶訂購的數量，計算每項產品的「小計（Subtotal）」及總金額，並在回應的最下方提供總金額。
   - 例如，客戶說：「我要一杯奶茶，兩個漢堡蛋，和一份總匯三明治」，你的回應應為：
     ```
     漢堡蛋 NT$50 x2 = NT$100
     總匯三明治 NT$70 x1 = NT$70
     奶茶 NT$15 x1 = NT$15
     總計：NT$100 + NT$70 + NT$15 = NT$185
     ```
"""

human_prompt = "麻煩給我兩個漢堡蛋，一份總匯三明治，以及兩杯奶茶。"

ai_prompt = """
好的！以下是您的點單內容：

漢堡蛋 NT$50 x2 = NT$100
總匯三明治 NT$70 x1 = NT$70
奶茶 NT$15 x1 = NT$15
總計：NT$100 + NT$70 + NT$15 = NT$185
"""

In [ ]:
# 利用 ChatPromptTemplate 規範 LLM 的回應內容
from langchain.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", human_prompt),
    ("ai", ai_prompt),
    ("human", "{human_input}")
])

# 測試 LLM 對於模板之輸入一切正常
chat_messages = chat_prompt.format(human_input="請給我一份漢堡蛋，兩杯奶茶。")
print(llm.invoke(chat_messages))

好的！以下是您的點單內容：

漢堡蛋 NT$50 x1 = NT$50
奶茶 NT$15 x2 = NT$30
總計：NT$50 + NT$30 = NT$80


In [ ]:
# 定義一個以 LLM 處理使用者輸入的函數
def llm_chat(input_text):
    # msg = llm.invoke(input_text)
    # 將使用者之輸入填入模板中
    chat_messages = chat_prompt.format(human_input=input_text)
    # 要求 LLM 回答
    msg = llm.invoke(chat_messages)

    return msg

## 安裝 ngrok 網頁伺服器相關套件

* 細節可以參考： https://pyngrok.readthedocs.io/en/latest/integrations.html#google-colaboratory

In [ ]:
# 安裝 pyngrok。一款專為 Python 打造的 ngrok 網頁伺服器。
!pip install pyngrok

In [ ]:
# 取得 ngrok 登入金鑰（Authtoken）並加入 Linux 系統中
# 登入 ngrok 主頁面： https://ngrok.com/
# 點擊左側欄 "Getting Started" > "Your Authtoken"
# 將畫面中央標示為 "Command Line" 的命令，拷貝貼上
# 金鑰（Authtoken）每人不同，請勿照抄別人的金鑰。
# 個人金鑰不會改變，請妥善保管。
!ngrok config add-authtoken 1TWJSwmSTyaHLvnOJhnjV1pFP5W_5XaVvtwaze1ACiQMDBdm7

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# 攔截 LINE 訊息

* 細節請看這個網站： https://github.com/line/line-bot-sdk-python

In [ ]:
# 安裝 Line Bot SDK
!pip install line-bot-sdk

In [ ]:
# 攔截 LINE 訊息主程式，可由 LINE Bot SDK 網站拷貝得到

# 引入 Flask 網頁伺服器存取套件
from flask import Flask, request, abort

# 引入 Line Bot SDK 相關套件
from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

# 引入 pyngrok 相關套件
import os
import threading
from pyngrok import ngrok

# 產生一個 Flask 物件，連上 Web 伺服器
app = Flask(__name__)

# 啟動 ngrok
# 將 ngrok 連往 Flask 預設埠號 5000
port = "5000"
# 取得 ngrok 公共 URL
public_url = ngrok.connect(port).public_url
# 印出 ngrok 公共 URL，並告知連往本地端埠號 5000
print(f" * ngrok tunnel '{public_url}' -> 'http://127.0.0.1:{port}'")
# 將本服務的網址，由 localhost，轉為公共的 URL
app.config["BASE_URL"] = public_url

# Channel Acess Token 類似登入帳號，Channel Secret 類似登入密碼
# 兩者請至 LINE Developer 網站，相應 Channel 內取得
configuration = Configuration(access_token='8dfPJoXVtadZ/Na124XnFDri+8PHZpFtqcrYt4li/G1smRd+50W4FoMOXn1TSMCPvdv6U4wdKyCEHT15s8blfHSbNTpbbMvWbUiXZToO4GwX+3fzGbZvkXeldBrHuKWP3bVAjN3mbUulo1vCqB4tnQdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('34e99337d24e559e542dc8f46690645e')

# 測試 ngrok 伺服器是否正常運作之函數
# 只要輸入公共 URL 於瀏覽器中，回傳 "Hello from ngrok!"，ngrok 就是正常運作
# 此函數不寫也可以，寫了可以方便日後測試
@app.route("/")
def index():
    return "Hello from ngrok!"

# 網頁封包處理函數，不要更動
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

# 網頁訊息處理函數，可以客製化
# event.message.text 就是使用者輸入的文字
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        # 將使用者輸入的訊息，送往大語言模型處理
        reply_message = llm_chat(event.message.text)

        # 回覆使用者輸入的訊息
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=reply_message)]
            )
        )

if __name__ == "__main__":
    # 以另一條執行緒將 Flask 啟動於背景，免得阻塞 Colab 使之無法操作
    threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

 * ngrok tunnel 'https://1ff0-34-23-234-74.ngrok-free.app' -> 'http://127.0.0.1:5000'
 * Serving Flask app '__main__'
 * Debug mode: off
